In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
data = pd.read_csv("SO_smsposter_postaggdata.csv")

In [5]:
subset = data[['ques_id', 'url', 'answers','comments']]
tuples = [tuple(x) for x in subset.to_numpy()]

In [19]:
# Loop through the dataset and extract the necessary data
data_dict = {'thread_id':[],'indv_id':[],'type':[],'text':[]}
for e in tuples:
    tid = e[0] # thread id
    url = e[1]
    answers = e[2].split("&")
    comments = e[3].split("&")
    
    #set params for scraping
    page = requests.get(url)
    print(page.status_code)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #extract question text
    question = soup.find('div', class_='post-text')
    q_data = question.get_text()
    q_data_fin = ''
    data_points = q_data.split('\n')
    for p in data_points:
        q_data_fin+=p
    #print(q_data_fin)
    data_dict['thread_id'].append(tid)
    data_dict['indv_id'].append(tid)
    data_dict['type'].append('question')
    data_dict['text'].append(q_data_fin)
    
    #extract the answers
    for ans in answers:
        aid = 'answer-'+ans
        answer_xml = soup.find('div',id=aid).find("div",class_='answercell post-layout--right').find('div',class_='post-text')
        #print(answer_xml.get_text())
        ans_text = answer_xml.get_text()
        data_dict['thread_id'].append(tid)
        data_dict['indv_id'].append(ans)
        data_dict['type'].append('answer')
        data_dict['text'].append(ans_text)
        
    #extract comments
    if 'NO_DATA' not in comments:
        print(comments)
        for comm in comments:
            cid = 'comment-'+comm
            #print(cid)
            comment_xml = soup.find('li',id=cid)
            #print(comment_xml)
            if comment_xml:
                print(cid)
                a = comment_xml.find_all('span',class_='comment-copy')
                #print(a[0].get_text())
                comm_text = a[0].get_text()
                data_dict['thread_id'].append(tid)
                data_dict['indv_id'].append(comm)
                data_dict['type'].append('comment')
                data_dict['text'].append(comm_text)
    
        
    print('collected data for:',tid)
    
final = pd.DataFrame(data_dict)

200
['106690990', '106676117', '106573506', '106566623', '106708583', '106667964']
comment-106690990
comment-106676117
comment-106573506
comment-106566623
comment-106708583
comment-106667964
collected data for: 58747823
200
['101052682', '106662140', '106660065', '101146238', '101101242', '101099934', '101086659', '101086651', '101065691', '101053829', '101053520', '101052889', '101052406']
comment-101052682
comment-106662140
comment-106660065
comment-101146238
comment-101101242
comment-101099934
comment-101086659
comment-101086651
comment-101065691
comment-101053829
comment-101053520
comment-101052889
comment-101052406
collected data for: 57277759
200
['95570930', '95545980', '95545929', '95545885', '95545705', '100298455', '95572221', '95570929']
comment-95570930
comment-95545980
comment-95545929
comment-95545885
comment-95545705
comment-100298455
comment-95572221
comment-95570929
collected data for: 54365906
200
collected data for: 58386069
200
['103205998', '100544257', '100415692'

In [20]:
final.to_csv("SO_smsposter_finaldata.csv")